# 모두의 말뭉치 - 비출판물 말뭉치
> https://kli.korean.go.kr/corpus/request/corpusList.do

In [4]:
import pandas as pd
import os
import json

In [3]:
df = pd.read_csv(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\NIKL_NP.csv')
df.head()

,Unnamed: 0,file_id,anno_level,category,title,author_id,author_age,author_occupation,author_sex,author_submission,author_handwriting,text_date,text_subclass,sentence
0,0,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어제는 아내의 생일이었다. 생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터...
1,1,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,"주된 메뉴는 스테이크와 낙지볶음, 미역국, 잡채, 소야 등이었다. 스테이크는 자주하..."
2,2,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,그런데 상상도 못한 일이 벌이지고 말았다. 보통 시즈닝이 되지 않은 원육을 사서 스...
3,3,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,앞면을 센불에 1분을 굽고 뒤집는 순간 방부제가 함께 구어진 것을 알았다. 아내의 ...
4,4,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어처구니 없는 상황이 발생한 것이다. 방부제가 센불에 녹아서 그런지 물처럼 흘러내렸...


In [5]:
df.isnull().sum()

Unnamed: 0             0
file_id                0
anno_level             0
category               0
title                  0
author_id              0
author_age             0
author_occupation     27
author_sex             0
author_submission      0
author_handwriting     0
text_date              0
text_subclass          0
sentence               0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [13]:
df[df['title']=='아내의 생일상']

,Unnamed: 0,file_id,anno_level,category,title,author_id,author_age,author_occupation,author_sex,author_submission,author_handwriting,text_date,text_subclass,sentence
0,0,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어제는 아내의 생일이었다. 생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터...
1,1,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,"주된 메뉴는 스테이크와 낙지볶음, 미역국, 잡채, 소야 등이었다. 스테이크는 자주하..."
2,2,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,그런데 상상도 못한 일이 벌이지고 말았다. 보통 시즈닝이 되지 않은 원육을 사서 스...
3,3,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,앞면을 센불에 1분을 굽고 뒤집는 순간 방부제가 함께 구어진 것을 알았다. 아내의 ...
4,4,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어처구니 없는 상황이 발생한 것이다. 방부제가 센불에 녹아서 그런지 물처럼 흘러내렸...
5,5,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,방부제가 묻은 부문만 제거 하고 다시 구울까 했는데 방부제에 절대 먹지 말라는 문구...
6,6,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,"하지만 정신을 추스리고 바로 다른 메뉴로 변경했다. 소야, 소세지 야채볶음... 아..."
7,7,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,음식은 성공적으로 완성이 되었다. 40번째를 맞이하는 아내의 생일은 성공적으로 준비...
8,8,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,오늘도 즐거운 하루가 되었으면 좋겠다. 생일이니까~
